# MODEL TRAINING

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('censusdata/adult.csv',names=['age','workclass','fnlwgt','education','education_num','marital-status','occupation','relationship','race','sex','capital_gain','capital_loss','hours-per-week','native-country','Target'])
df

,age,workclass,fnlwgt,education,education_num,marital-status,occupation,relationship,race,sex,capital_gain,capital_loss,hours-per-week,native-country,Target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
X = df.drop(labels=['Target'],axis=1)
Y = df[['Target']]

In [4]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [6]:
numerical_cols

Index(['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
       'hours-per-week'],
      dtype='object')

In [38]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer



In [40]:
# Numerical Pipeline
num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

# Categorical Pipeline
cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(drop='if_binary'))
    ]
)

# Combine the numerical and categorical pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num_pipeline', num_pipeline, numerical_cols),
        ('cat_pipeline', cat_pipeline, categorical_cols)
    ],
    remainder='passthrough'
)

# Create the final pipeline
pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor)
        # Add additional steps as needed (e.g., model)
    ]
)

In [41]:
## Train Test Split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)

In [51]:
'''
x=pd.DataFrame(X_train_encoded)
x
X_train_dense = X_train_encoded.toarray()
len(X_train_dense[0])
'''

107

In [52]:
# Fit and transform the training data
X_train_encoded = pipeline.fit_transform(X_train)
X_train_columns = pipeline['preprocessor'].get_feature_names_out()
X_train_dense = X_train_encoded.toarray()
X_train = pd.DataFrame(X_train_dense, columns=X_train_columns)

# Transform the test data
X_test_encoded = pipeline.transform(X_test)
X_test_columns = pipeline['preprocessor'].get_feature_names_out()
X_test_dense = X_test_encoded.toarray()
X_test = pd.DataFrame(X_test_dense, columns=X_test_columns)

In [53]:
X_train.head()

,num_pipeline__age,num_pipeline__fnlwgt,num_pipeline__education_num,num_pipeline__capital_gain,num_pipeline__capital_loss,num_pipeline__hours-per-week,cat_pipeline__workclass_ ?,cat_pipeline__workclass_ Federal-gov,cat_pipeline__workclass_ Local-gov,cat_pipeline__workclass_ Never-worked,...,cat_pipeline__native-country_ Portugal,cat_pipeline__native-country_ Puerto-Rico,cat_pipeline__native-country_ Scotland,cat_pipeline__native-country_ South,cat_pipeline__native-country_ Taiwan,cat_pipeline__native-country_ Thailand,cat_pipeline__native-country_ Trinadad&Tobago,cat_pipeline__native-country_ United-States,cat_pipeline__native-country_ Vietnam,cat_pipeline__native-country_ Yugoslavia
0,0.762703,0.035050,-0.423183,-0.145994,-0.219309,-0.031430,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.850415,-0.527734,-0.423183,-0.145994,-0.219309,0.375095,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.396085,-0.187216,-1.200011,-0.145994,-0.219309,1.594670,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.495939,4.432379,1.130472,-0.145994,-0.219309,1.594670,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.129321,-0.014657,1.130472,-0.145994,3.251875,-0.194040,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [54]:
## Model Training
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [56]:
log = LogisticRegression()
model=log.fit(X_train,y_train)

e:\MLPROJECTS\CensusProject\venv\lib\site-packages\scikit_learn-1.2.2-py3.8-win-amd64.egg\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
e:\MLPROJECTS\CensusProject\venv\lib\site-packages\scikit_learn-1.2.2-py3.8-win-amd64.egg\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [57]:
y_pred = model.predict(X_test)

In [59]:
y_pred

array([' <=50K', ' <=50K', ' >50K', ..., ' <=50K', ' >50K', ' <=50K'],
      dtype=object)

In [60]:
accuracy_score(y_test,y_pred)

0.8576341972730622

In [61]:
from sklearn.metrics import confusion_matrix

In [62]:
cm = confusion_matrix(y_test,y_pred)
cm

array([[5809,  405],
       [ 754, 1173]], dtype=int64)

In [63]:
from sklearn.tree import DecisionTreeClassifier

In [64]:
dtc = DecisionTreeClassifier()
model = dtc.fit(X_train,y_train)

In [66]:
y_pred = model.predict(X_test)

In [67]:
y_pred

array([' <=50K', ' <=50K', ' >50K', ..., ' <=50K', ' >50K', ' <=50K'],
      dtype=object)

In [68]:
accuracy_score(y_test,y_pred)

0.8242230684191131

In [69]:
cm = confusion_matrix(y_test,y_pred)
cm

array([[5450,  764],
       [ 667, 1260]], dtype=int64)

In [76]:
## model training
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [77]:
import numpy as np
def evaluate_model(true,predicted):
    accuracy = accuracy_score(true,predicted)
    cm = confusion_matrix(true,predicted)
    return accuracy, cm

In [78]:
## Train mutiple models

models = {
    'LogisticRegression' : LogisticRegression(),
    'DecisionTreeClassifier' : DecisionTreeClassifier(),
    'RandomForestClassifier' : RandomForestClassifier(),
}

model_list=[]
accuracy_list=[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    # make prediction
    y_pred = model.predict(X_test)

    accuracy,cm=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print('Accuracy score: ', accuracy)
    print('Confusion matrix: ')
    print(cm)

    accuracy_list.append(accuracy)

    print('='*40)
    print('\n')

e:\MLPROJECTS\CensusProject\venv\lib\site-packages\scikit_learn-1.2.2-py3.8-win-amd64.egg\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
e:\MLPROJECTS\CensusProject\venv\lib\site-packages\scikit_learn-1.2.2-py3.8-win-amd64.egg\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression
Model Training Performance
Accuracy score:  0.8576341972730622
Confusion matrix: 
[[5809  405]
 [ 754 1173]]


DecisionTreeClassifier
Model Training Performance
Accuracy score:  0.8221348728657413
Confusion matrix: 
[[5450  764]
 [ 684 1243]]




C:\Users\Kanav Gupta\AppData\Local\Temp\ipykernel_13120\3403633882.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train,y_train)


RandomForestClassifier
Model Training Performance
Accuracy score:  0.8608279081193957
Confusion matrix: 
[[5791  423]
 [ 710 1217]]




In [79]:
model_list

['LogisticRegression', 'DecisionTreeClassifier', 'RandomForestClassifier']